In [1]:
#dependencies
import re
from bs4 import BeautifulSoup 
from splinter import Browser
import requests
import pandas as pd
from selenium import webdriver 
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as BraveService
from webdriver_manager.core.utils import ChromeType
import json
import time
from tqdm.notebook import tqdm 

<h4> Start cells here if partial FIPS already available in {state}_fips_scraped file

In [2]:
fips_check_scraped = pd.read_csv("../../../data/state_data/geo/geo_fips/23/DE_fips_scraped.csv") 
fips_check_scraped  = fips_check_scraped.loc[:, ~fips_check_scraped.columns.str.contains('^Unnamed')] 
fips_check_scraped.head() 

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,5512588105,DE,"3821 Lancaster Pike, Wilmington, DE",39.757096,-75.595665,1.000301e+14
1,7963857007,DE,"2502 W. 6TH ST, WILMINGTON, DE",39.753040,-75.577856,1.000300e+14
2,5522267710,DE,"222 VALLEY RD, WILMINGTON, DE",39.726703,-75.582135,1.000301e+14
3,6561867002,DE,"19 Germay Dr, WILMINGTON, DE",39.728914,-75.578340,1.000301e+14
4,6162427002,DE,"6263 Kellers Church Road, PIPERSVILLE, DE",40.392992,-75.153345,NaN


In [3]:
#Latitude, Longitude, not included
fips_check_scraped_notna = fips_check_scraped[fips_check_scraped["Lat"].notna()] # check to make sure no nulls in coords
fips_check_scraped_notna.head() 

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,5512588105,DE,"3821 Lancaster Pike, Wilmington, DE",39.757096,-75.595665,1.000301e+14
1,7963857007,DE,"2502 W. 6TH ST, WILMINGTON, DE",39.753040,-75.577856,1.000300e+14
2,5522267710,DE,"222 VALLEY RD, WILMINGTON, DE",39.726703,-75.582135,1.000301e+14
3,6561867002,DE,"19 Germay Dr, WILMINGTON, DE",39.728914,-75.578340,1.000301e+14
4,6162427002,DE,"6263 Kellers Church Road, PIPERSVILLE, DE",40.392992,-75.153345,NaN


In [4]:
#FIPS not included, of Lat, Long available
fips_check_scraped_null = fips_check_scraped_notna[fips_check_scraped_notna["FIPS_z"].isnull()] # check to make sure no nulls in coords
fips_check_scraped_null #28093

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
4,6162427002,DE,"6263 Kellers Church Road, PIPERSVILLE, DE",40.392992,-75.153345,NaN
1578,1745487302,DE,"6300 PHILADELPHIA PIKE, CLAYMONT, DE",39.812609,-75.432185,NaN
2629,9523837105,DE,"30003 Pawley Island Ct, DAGSBORO, DE",33.433224,-79.121431,NaN


<h4>FIPS Script

In [5]:
#if FIPS_z file 
lat = fips_check_scraped_null["Lat"].astype(float).astype(str).to_list()
long = fips_check_scraped_null["Long"].astype(float).astype(str).to_list()

In [6]:
#before running script, check for any NaN within Lat, Long fields. 
def sleep():
    time.sleep(1)
option = webdriver.ChromeOptions()
option.binary_location = "/Applications/Brave Browser.app/Contents/MacOS/Brave Browser"
driver = Browser('chrome', options=option, headless=False)    
url = 'https://geocoding.geo.census.gov/geocoder/geographies/coordinates?form'
driver.visit(url)

not_found = []
geo_id = []

i=0
for i in tqdm(range(len(lat))):
    #while i < len(lat):
        driver.find_by_id('geoCoordX').clear()
        driver.find_by_id('geoCoordY').clear()
        driver.find_by_id('geoCoordX').fill(long[i])
        driver.find_by_id('geoCoordY').fill(lat[i])
        driver.find_by_id('geoCoordsSubmit').click()
        #sleep()
        #html = driver.html
        soup = BeautifulSoup(driver.html, 'html.parser')


        results = soup.find_all('div', id='geoCoordsJsonOutput')
        for result in results:
            result_text = result.text

            result_digits = re.findall('\d+', result_text)
            startswith = [s for s in result_digits if s.startswith('10')] #DE
            fips_elements =[element for element in startswith if len(element)>=15]
            
            try:
                #print(f"{i},{lat[i]},{long[i]} is {fips_elements}")
                geo_id.append(fips_elements)
            except (KeyError, IndexError):
                #print(f"{i},{lat[i]},{long[i]} Not Found")
                not_found.append(i)
                geo_id.append(None)  
            except (TimeoutException):
                driver.navigate.refresh()
    
        #sleep()
        i=i+1

  0%|          | 0/3 [00:00<?, ?it/s]

coordinates wtih more than one return:
385,37.514070000000004,-77.35472 is ['514070000000004', '510872015041003'] 
1178,37.516454700000004,-77.8100575 is ['516454700000004', '511455001022014'] 
2660,37.519462299999994,-77.340384 is ['519462299999994', '510872014061012']
second option reference FIPS code, first option references LatCoord, update fields in DataFrame

https://iqss.github.io/dss-webscrape/filling-in-web-forms.html

In [7]:
geo_id_list =geo_id 

Length Check

In [8]:
print(len(geo_id_list))

3


Matching FIPS to DataFrame

In [9]:
fips_fill = fips_check_scraped_null

In [10]:
fips_fill["FIPS_z"] = geo_id_list
fips_fill

/var/folders/vy/qdp8xcfj4s3dd5c6d1xp_y3w0000gn/T/ipykernel_15182/597263667.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill["FIPS_z"] = geo_id_list


,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
4,6162427002,DE,"6263 Kellers Church Road, PIPERSVILLE, DE",40.392992,-75.153345,[]
1578,1745487302,DE,"6300 PHILADELPHIA PIKE, CLAYMONT, DE",39.812609,-75.432185,[]
2629,9523837105,DE,"30003 Pawley Island Ct, DAGSBORO, DE",33.433224,-79.121431,[]


Remove brackets

In [11]:
fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]
fips_fill

/var/folders/vy/qdp8xcfj4s3dd5c6d1xp_y3w0000gn/T/ipykernel_15182/1219885402.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]


,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
4,6162427002,DE,"6263 Kellers Church Road, PIPERSVILLE, DE",40.392992,-75.153345,NaN
1578,1745487302,DE,"6300 PHILADELPHIA PIKE, CLAYMONT, DE",39.812609,-75.432185,NaN
2629,9523837105,DE,"30003 Pawley Island Ct, DAGSBORO, DE",33.433224,-79.121431,NaN


In [12]:
fips_fill.loc[fips_fill["FIPS_z"].isnull()] #check #13

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
4,6162427002,DE,"6263 Kellers Church Road, PIPERSVILLE, DE",40.392992,-75.153345,NaN
1578,1745487302,DE,"6300 PHILADELPHIA PIKE, CLAYMONT, DE",39.812609,-75.432185,NaN
2629,9523837105,DE,"30003 Pawley Island Ct, DAGSBORO, DE",33.433224,-79.121431,NaN


Parse for Merge

In [13]:
cols= ["LoanNumber", "FIPS_z"]
fips_fill_parsed = fips_fill[cols]
fips_fill_parsed

,LoanNumber,FIPS_z
4,6162427002,NaN
1578,1745487302,NaN
2629,9523837105,NaN


Read in base data

In [14]:

#if from FIPS file
fips_check = pd.read_csv("../../../data/state_data/geo/geo_fips/23/DE_fips_scraped.csv") 
#fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1", "Unnamed: 0"], axis =1) 
fips_check.head()

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,201227,5512588105,DE,"3821 Lancaster Pike, Wilmington, DE",39.757096,-75.595665,1.000301e+14
1,201228,7963857007,DE,"2502 W. 6TH ST, WILMINGTON, DE",39.753040,-75.577856,1.000300e+14
2,201229,5522267710,DE,"222 VALLEY RD, WILMINGTON, DE",39.726703,-75.582135,1.000301e+14
3,201230,6561867002,DE,"19 Germay Dr, WILMINGTON, DE",39.728914,-75.578340,1.000301e+14
4,201231,6162427002,DE,"6263 Kellers Church Road, PIPERSVILLE, DE",40.392992,-75.153345,NaN


Merge with MainFrame

In [15]:
fips_merge = fips_check.merge(fips_fill_parsed, on="LoanNumber", how="left")
fips_merge

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z_x,FIPS_z_y
0,201227,5512588105,DE,"3821 Lancaster Pike, Wilmington, DE",39.757096,-75.595665,1.000301e+14,NaN
1,201228,7963857007,DE,"2502 W. 6TH ST, WILMINGTON, DE",39.753040,-75.577856,1.000300e+14,NaN
2,201229,5522267710,DE,"222 VALLEY RD, WILMINGTON, DE",39.726703,-75.582135,1.000301e+14,NaN
3,201230,6561867002,DE,"19 Germay Dr, WILMINGTON, DE",39.728914,-75.578340,1.000301e+14,NaN
4,201231,6162427002,DE,"6263 Kellers Church Road, PIPERSVILLE, DE",40.392992,-75.153345,NaN,NaN
...,...,...,...,...,...,...,...,...
2985,204212,7925667008,DE,"1 CENTURIAN DR Suite 106, NEWARK, DE",39.695892,-75.656759,1.000301e+14,NaN
2986,204213,8190337202,DE,"824 Philadelphia Pike, Wilmington, DE",39.772827,-75.495663,1.000301e+14,NaN
2987,204214,9093838307,DE,"22586 Sussex Hwy, Seaford, DE",38.670838,-75.593801,1.000505e+14,NaN
2988,204215,9156708410,DE,"1 Winston Ave, Wilmington, DE",39.728615,-75.586426,1.000301e+14,NaN


<b>Fill FIPS Columns - if FIPS file

In [16]:
fips_merge["FIPS_z"] = fips_merge["FIPS_z_y"].fillna(fips_merge["FIPS_z_x"])
fips_merge.head()

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z_x,FIPS_z_y,FIPS_z
0,201227,5512588105,DE,"3821 Lancaster Pike, Wilmington, DE",39.757096,-75.595665,1.000301e+14,NaN,1.000301e+14
1,201228,7963857007,DE,"2502 W. 6TH ST, WILMINGTON, DE",39.753040,-75.577856,1.000300e+14,NaN,1.000300e+14
2,201229,5522267710,DE,"222 VALLEY RD, WILMINGTON, DE",39.726703,-75.582135,1.000301e+14,NaN,1.000301e+14
3,201230,6561867002,DE,"19 Germay Dr, WILMINGTON, DE",39.728914,-75.578340,1.000301e+14,NaN,1.000301e+14
4,201231,6162427002,DE,"6263 Kellers Church Road, PIPERSVILLE, DE",40.392992,-75.153345,NaN,NaN,NaN


Drop Extra FIPS Columns

In [17]:
fips_merge_drop = fips_merge.drop(["FIPS_z_x", "FIPS_z_y"], axis=1)
fips_merge_drop = fips_merge_drop.drop(["Unnamed: 0"], axis=1)
fips_merge_drop

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,5512588105,DE,"3821 Lancaster Pike, Wilmington, DE",39.757096,-75.595665,1.000301e+14
1,7963857007,DE,"2502 W. 6TH ST, WILMINGTON, DE",39.753040,-75.577856,1.000300e+14
2,5522267710,DE,"222 VALLEY RD, WILMINGTON, DE",39.726703,-75.582135,1.000301e+14
3,6561867002,DE,"19 Germay Dr, WILMINGTON, DE",39.728914,-75.578340,1.000301e+14
4,6162427002,DE,"6263 Kellers Church Road, PIPERSVILLE, DE",40.392992,-75.153345,NaN
...,...,...,...,...,...,...
2985,7925667008,DE,"1 CENTURIAN DR Suite 106, NEWARK, DE",39.695892,-75.656759,1.000301e+14
2986,8190337202,DE,"824 Philadelphia Pike, Wilmington, DE",39.772827,-75.495663,1.000301e+14
2987,9093838307,DE,"22586 Sussex Hwy, Seaford, DE",38.670838,-75.593801,1.000505e+14
2988,9156708410,DE,"1 Winston Ave, Wilmington, DE",39.728615,-75.586426,1.000301e+14


In [18]:
fips_merge_drop.loc[fips_merge_drop["FIPS_z"].isnull()] #13

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
4,6162427002,DE,"6263 Kellers Church Road, PIPERSVILLE, DE",40.392992,-75.153345,NaN
1578,1745487302,DE,"6300 PHILADELPHIA PIKE, CLAYMONT, DE",39.812609,-75.432185,NaN
2629,9523837105,DE,"30003 Pawley Island Ct, DAGSBORO, DE",33.433224,-79.121431,NaN


Convert to File

In [19]:
fips_DE = fips_merge_drop

In [20]:
fips_DE.to_csv("../../../data/state_data/geo/geo_fips/23/DE_fips_scraped.csv") 